In [1]:
import numpy as np
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
"""
Conway's Game of  Life

Game Rules:
At each itteration:
1. Any alive cell with less than two alive neighbours, dies of loneliness.
2. Any alive cell with more than two alive neighbours, dies of overpopulation.
3. Any cell with exactly three alive neighbours becomes an alive cell.
4. Any cell with exactly two alive neighbours stays in the current status in the next itteration.
"""

class Environment:
  def __init__(self, rows, cols):
    self.rows = rows
    self. cols = cols
    self.world = np.zeros((rows,cols))

  def born_cell(self, row, col):
    self.world[row,col] = 1

  def kill_cell(self, row, col):
    self.world[row,col] = 0

  def invert_cell(self,row,col):
    self.world[row,col] = int(not self.world[row,col])

  
  def cell_neighbourhood(self, row, col):
    #lists the cells adjacent to a single cell, no matter if they are alive or dead
    #right neighbour
    if (col == self.world.shape[1]-1):
      right = []
    else:      
      right = [row,col+1]
    
    #down neighbour
    if (row == self.world.shape[0]-1):
      down = []
    else:
      down = [row+1, col]
    
    #left neighbour
    if (col == 0):
      left = []
    else:
      left = [row,col-1]
    
    #up neighbour
    if (row == 0):
      up = []
    else:
      up = [row-1, col]

    #up left neighbour
    if (col == 0 or row ==0):
      ul = []
    else:
      ul = [row-1, col-1]

    #down right neighbour
    if (col == self.world.shape[1]-1 or row == self.world.shape[0]-1):
      dr = []
    else:
      dr = [row+1, col+1]

    #up right neighbour
    if (col == self.world.shape[1]-1 or row == 0):
      ur = []
    else: ur = [row-1, col+1]

    #down left neighbour
    if (col == 0 or row == self.world.shape[0]-1):
      dl = []
    else:
      dl = [row+1, col-1]

    return [ul, up, ur, right, dr, down, dl, left]

  
  def cell_neighbours(self, row, col):
    #returns the number of alive neighbours of a cell
    neigs = 0
    for n in self.cell_neighbourhood(row, col):
      if (len(n) != 0):
        neigs += self.world[n[0],n[1]]
    return neigs


  def cell_next_state(self, row, col):
    #based on how many alive neighbours a cell has, calculates the status of the cell in the next itteration
    #status: 1 (alive) or 0 (dead)
    if (self.cell_neighbours(row, col) == 2):
      return self.world[row,col]
    elif (self.cell_neighbours(row, col) == 3):
      return 1
    else:
      return 0
  
  def update_world(self):
    #calculates the next status of each cell and redraw the world with the updated values
    next_world = np.zeros(self.world.shape)
    for row in range(self.world.shape[0]):
      for col in range(self.world.shape[1]):
        next_world[row,col] = self.cell_next_state(row, col)
    self.world = next_world
    

#World Creation:
world_rows = 75
world_cols = 120
amb = Environment(world_rows,world_cols)


#Creating alive cells randomly
for r in range(amb.world.shape[0]):
    for c in range(amb.world.shape[1]):
        if (np.random.random() <= .3):
            amb.born_cell(r, c)


#Creates GUI
#GUI credits to Cribber, at Stack Overflow
grid_width = 1200
grid_height = 750
gridDisplay = pygame.display.set_mode((grid_width, grid_height))
pygame.display.get_surface().fill((0, 0, 0))  # background

# we use the sizes to draw as well as to do our "steps" in the loops. 
grid_node_width = grid_width/world_cols-1
grid_node_height = grid_height/world_rows-1

def createSquare(x, y, color):
    pygame.draw.rect(gridDisplay, color, [x, y, grid_node_width, grid_node_height ])

def visualizeGrid():
    y = 0  # we start at the top of the screen
    for row in amb.world:
        x = 0 # for every row we start at the left of the screen again
        for item in row:
            if item == 0:
                createSquare(x, y, (0, 0, 0))
            else:
                createSquare(x, y, (0, 127, 255))

            x += grid_node_width+1 # for ever item/number in that row we move one "step" to the right
        y += grid_node_height+1   # for every new row we move one "step" downwards
    pygame.display.update()
    

#Initializes GUI
visualizeGrid()

#Updates world and GUI until the user closes the window or presses ESC
while True:
    pygame.time.delay(150)
    amb.update_world()
    visualizeGrid()
    event = pygame.event.get()
    if (len(event) > 0) and (event[0].type == 32787): #Tests if the window has been closed
        break
    if (len(event) > 0) and (event[0].type == 768) and (event[0].dict['key'] == 27): #Tests if ESC key has been pressed
        break

pygame.quit() #Ends the program